In [1]:
import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV

import theano
import theano.tensor as T

from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.updates import adam
from nolearn.lasagne import NeuralNet
from lasagne.nonlinearities import softmax
from lasagne.nonlinearities import sigmoid


from lasagne.layers import get_output, InputLayer, DenseLayer, Upscale1DLayer, ReshapeLayer, DropoutLayer
from lasagne.nonlinearities import rectify, leaky_rectify, tanh
from lasagne.updates import nesterov_momentum
from lasagne.objectives import categorical_crossentropy
from nolearn.lasagne import NeuralNet, BatchIterator, PrintLayerInfo

from lasagne.layers import Conv1DLayer as Conv1DLayerSlow
from lasagne.layers import MaxPool1DLayer as MaxPool1DLayerSlow

SkipTest: You are importing theano.sandbox.cuda. This is the old GPU back-end and is removed from Theano. Use Theano 0.9 to use it. Even better, transition to the new GPU back-end! See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

In [ ]:

try:
    from lasagne.layers.cuda_convnet import Conv1DCCLayer 
    from lasagne.layers.cuda_convnet import MaxPool1DCCLayer 
    print('Using cuda_convnet (faster)')
except:
    from lasagne.layers import Conv1DLayer
    from lasagne.layers import MaxPool1DLayer
    print('Using lasagne.layers (slower)')

In [ ]:
t = pd.read_csv("./data.csv")

In [ ]:
__type = "math"#sys.argv[1]
t[__type] =  t["labels"].str.contains(__type)
t[__type] = pd.to_numeric(t[__type], downcast="signed")

In [ ]:
feature = "problem-statement"
output = t[[__type,feature]]
output = output.dropna()

output[__type] = output[__type].astype(int)

df = pd.DataFrame(columns=[__type, feature])

a = output.loc[output[__type] == 1.0]
b = output.loc[output[__type] == 0.0]
b = b.sample(n=len(a), random_state = 42)

In [ ]:
output = a.append(b)


In [ ]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words="english",analyzer = 'word')

l = output[feature].tolist()
res = []
regex = re.compile('^a-zA-Z0-9')
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        res.append(clean_s(l[i]))
    except:
        res.append("")

X = vectorizer.fit_transform(res)

In [ ]:
#model = linear_model.RidgeCV(alphas = [0.1, 1.0, 10])
#parameters = {}
#model = linear_model.Lasso(alpha=0.0001)
#model = linear_model.SGDClassifier(loss = 'hinge', penalty='l2')
#parameters = {}
#model = GaussianNB()

### -->logistic regression
#model = linear_model.LogisticRegression(class_weight= 'balanced')
#parameters = {'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.1, 1, 10,100], 'max_iter': [100,1000]}

### -> logistic regression
#model = LinearSVC()
#model = MLPClassifier(solver='lbfgs',  hidden_layer_sizes= (100,100,100), random_state=1)
#parameters={}
#parameters = {'C':[ 0.1, 1, 10,100]}
### --> randomforest
#model = RandomForestClassifier(class_weight = 'balanced')
#parameters={}
### --> random forest


#model = AdaBoostClassifier(n_estimators=100)
#parameters= {'n_estimators': (10,100,1000)}
#### ---> SVC
#model = svm.SVC()
#parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[ 1, 10]}
### ---> SVC 
#clf = GridSearchCV(model, parameters)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,output[__type], test_size=0.2,random_state=42)
print X_train.toarray().shape

In [ ]:
pad_in = 'full'
pad_out = 'full'
layers0 = [
    # layer dealing with the input data
    (InputLayer, {'shape': (None, 1, X_train.shape[1] )}),

    # first stage of our convolutional layers
    (Conv1DLayer, {'num_filters': 16, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 16, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 16, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 16, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 16, 'filter_size': 4, 'pad':'full'}),
    (MaxPool1DLayer, {'pool_size': 2}),

    # second stage of our convolutional layers
    (Conv1DLayer, {'num_filters': 28, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 28, 'filter_size': 4, 'pad':'full'}),
    (Conv1DLayer, {'num_filters': 28, 'filter_size': 4, 'pad':'full'}),
    (MaxPool1DLayer, {'pool_size': 2}),

    # two dense layers with dropout
    (DenseLayer, {'num_units': 64}),
    (DropoutLayer, {}),
    (DenseLayer, {'num_units': 64}),

    # the output layer
    (DenseLayer, {'num_units': 1, 'nonlinearity': sigmoid}),
]
ae = NeuralNet(
    layers=layers0,
    max_epochs=20,
    
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.975,
    
    regression=True,
    verbose=1
)
x=X_train.toarray().reshape((X_train.shape[0], 1, X_train.shape[1]))
print X_train.shape
print y_train.shape
ae.fit(x, y_train)

In [ ]:
# for neural network only
net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden1', layers.DenseLayer),
        ('hidden2', layers.DenseLayer),
        ('hidden3', layers.DenseLayer),
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        #('hidden6', layers.DenseLayer),
        #('hidden7', layers.DenseLayer),
        #('hidden8', layers.DenseLayer),
        #('hidden9', layers.DenseLayer),
        #('hidden10', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, X_train.shape[1]), 
    hidden1_num_units=1000,  # number of units in hidden layer
    hidden2_num_units=1000,
    hidden3_num_units=1000,
    hidden4_num_units=1000,
    hidden5_num_units=1000,
   # hidden6_num_units=100,
   # hidden7_num_units=100,
   # hidden8_num_units=100,
   # hidden9_num_units=100,
   # hidden10_num_units=100,
    output_nonlinearity=sigmoid,  # output layer uses identity function
    output_num_units=1,  # 1 target values

    # optimization method:
    update=adam,
    update_learning_rate=0.001,
    #update_momentum=0.9,
    #classification = True,
    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=200,  # we want to train this many epochs
    verbose=1,
    )
net1.fit(X_train.toarray(), y_train) 
        
# ------>end 

#clf.fit(X_train, y_train)
#print sorted(clf.cv_results_.keys())

y_pred = net1.predict(X_test.toarray())
y_pred = [round(value) for value in y_pred]

print __type
print classification_report(y_test, y_pred)